In [1]:
import pandas as pd
import time
from datetime import datetime
import math
import ccxt
import json

In [2]:
with open('binance.json') as f:
    binance_api = json.load(f)

In [3]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)

exchange = exchange_class({
     "apiKey": "PasteBINANCEAPIKEY",
    "secret": "PasteSECRETKEY",
})

In [4]:
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets if market['type']=='spot']

In [5]:
def create_chain(base_currency, token1, token2, buy_t1, buy_t2_by_t1, sell_t2, res):   
    return {
        'base': base_currency, # USDT                 
        'token1':token1,      # BTC                 
        'token2':token2,      # ETH       
        'buy_t1': buy_t1,     
        'buy_t2_by_t1': buy_t2_by_t1,       
        'sell_t2': sell_t2,
        'Result' : res
    }
    

def get_chains(market_symbols, quotes, base_currency='USDT'):
    chains = []

    for symbol in market_symbols:
        try:
            token1 = symbol.split('/')[0] 
            token2 = symbol.split('/')[1]
            #ищем промежуточное звено треугольника и проверяем результат в 2 стороны
            if (token1!=base_currency and token2!=base_currency):
                start = quotes[f'{token1}/{base_currency}']
                middle = quotes[f'{token1}/{token2}']
                finish = quotes[f'{token2}/{base_currency}']
                if  start['ask']!=0 and start['bid']!=0 and middle['ask']!=0 and middle['bid']!=0 and finish['ask']!=0 and finish['bid']!=0:
                    #USDT-ETH-BTC
                    chains.append(
                                create_chain(
                                    base_currency, #USDT
                                    token1,        #ETH
                                    token2,        #BTC
                                    start['ask'],  #ETH/USDT ASK
                                    middle['bid'], #ETH/BTC BID
                                    finish['bid'], #BTC/USDT BID
                                    float(100.0*middle['bid']*finish['bid']/start['ask']-100.0)
                                )
                    )
                    #USDT-BTC-ETH
                    chains.append(
                                create_chain(
                                    base_currency, #USDT
                                    token2,        #BTC
                                    token1,        #ETH
                                    finish['ask'], #BTC/USDT ASK
                                    middle['ask'], ##ETH/BTC ASK
                                    start['bid'],  #ETH/USDT BID
                                    float(100.0*start['bid']/finish['ask']/middle['ask']-100.0)
                                )
                   )

        except KeyError:
            pass
    return pd.DataFrame(chains)


In [ ]:
while(1):    
    quotes = exchange.fetchBidsAsks()
    chains = get_chains(market_symbols, quotes, 'USDT')
    sorted_chains = chains.sort_values(by='Result', ascending=False)
    profit = sorted_chains.loc[sorted_chains['Result']>0.1]
    if not profit.empty:
        print(f"Time-{datetime.now().strftime('%H:%M:%S')}")
        print(profit)
    time.sleep(1)

Time-13:43:47
      base token1 token2    buy_t1  buy_t2_by_t1  sell_t2    Result
1519  USDT    BTC  VOXEL  22798.93      0.000018   0.4108  0.102164
Time-13:45:55
     base token1 token2   buy_t1  buy_t2_by_t1   sell_t2    Result
359  USDT    TRX    WIN  0.06868      0.002288  0.000157  0.101922
Time-13:46:03
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
359  USDT    TRX    WIN  0.0687      0.002293  0.000158  0.108488
Time-13:46:06
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
880  USDT    JUV    BTC    6.78      0.000298  22811.48  0.229202
Time-13:46:57
     base token1 token2   buy_t1  buy_t2_by_t1  sell_t2    Result
359  USDT    TRX    WIN  0.06874      0.002318  0.00016  0.100841
Time-13:46:58
     base token1 token2   buy_t1  buy_t2_by_t1  sell_t2    Result
359  USDT    TRX    WIN  0.06874      0.002318  0.00016  0.100841
Time-13:48:18
      base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
1753  USDT   BUSD   NEXO  0.9998      

Time-13:49:45
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
822  USDT   HARD    BTC  0.2641      0.000012  22855.48  0.127945
Time-13:49:47
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
822  USDT   HARD    BTC  0.2641      0.000012  22854.12  0.121987
Time-13:49:48
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
822  USDT   HARD    BTC  0.2641      0.000012  22852.28  0.113926
Time-13:49:49
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
822  USDT   HARD    BTC  0.2641      0.000012  22853.19  0.117913
Time-13:49:52
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2   Result
822  USDT   HARD    BTC  0.2641      0.000012  22849.48  0.10166
Time-13:49:59
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
347  USDT   USDC    WIN  0.9998      0.000164  0.000164  0.142203
359  USDT    TRX    WIN  0.0687      0.002383  0.000164  0.114775
Time-13:50:01
     base token1 token2  buy_t1  buy_t2_by_t1 

In [ ]:
#token1 - token2
100.0*0.674300*1.0008/0.67410-100.0

In [ ]:
#token2 - token1
100.0*0.513/19727.1600/0.000026-100.0